In [1]:
# import statements
import numpy as np
import requests
from bs4 import BeautifulSoup, Comment
import time
import csv
from collections import defaultdict
import pickle
import random
import pandas as pd
from util.client import Nba_Season
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

TEAM_NAME_TO_ABR = {
        "ATLANTA HAWKS": 'ATL',
        "BOSTON CELTICS": 'BOS',
        "BROOKLYN NETS": 'BRK',
        "CHARLOTTE HORNETS": 'CHO',
        "CHICAGO BULLS": 'CHI',
        "CLEVELAND CAVALIERS": 'CLE',
        "DALLAS MAVERICKS": 'DAL',
        "DENVER NUGGETS": 'DEN',
        "DETROIT PISTONS": 'DET',
        "GOLDEN STATE WARRIORS": 'GSW',
        "HOUSTON ROCKETS": 'HOU',
        "INDIANA PACERS": 'IND',
        "LOS ANGELES CLIPPERS": 'LAC',
        "LA CLIPPERS": 'LAC',
        "LOS ANGELES LAKERS": 'LAL',
        "LA LAKERS": 'LAL',
        "MEMPHIS GRIZZLIES": 'MEM',
        "MIAMI HEAT": 'MIA',
        "MILWAUKEE BUCKS": 'MIL',
        "MINNESOTA TIMBERWOLVES": 'MIN',
        "NEW ORLEANS PELICANS": 'NOP',
        "NEW YORK KNICKS": 'NYK',
        "OKLAHOMA CITY THUNDER": 'OKC',
        "ORLANDO MAGIC": 'ORL',
        "PHILADELPHIA 76ERS": 'PHI',
        "PHOENIX SUNS": 'PHO',
        "PORTLAND TRAIL BLAZERS": 'POR',
        "SACRAMENTO KINGS": 'SAC',
        "SAN ANTONIO SPURS": 'SAS',
        "TORONTO RAPTORS": 'TOR',
        "UTAH JAZZ": 'UTA',
        "WASHINGTON WIZARDS": 'WAS',

        # DEPRECATED TEAMS
        # "CHARLOTTE BOBCATS": 'CHA',
        # "KANSAS CITY KINGS": 'KCK',
        # "NEW JERSEY NETS": 'NJN',
        # "NEW ORLEANS HORNETS": 'NOH',
        # "NEW ORLEANS/OKLAHOMA CITY HORNETS": 'NOK',
        # "SEATTLE SUPERSONICS": 'SEA',
        # "ST. LOUIS HAWKS": 'STL',
        # "VANCOUVER GRIZZLIES": 'VAN',
        # "WASHINGTON BULLETS": 'WSB',
    }

TEAM_ABR_TO_URL = {
        "ATL": 'https://www.nba.com/stats/team/1610612737',
        "BOS": 'https://www.nba.com/stats/team/1610612738',
        "BRK": 'https://www.nba.com/stats/team/1610612751',
        "CHO": 'https://www.nba.com/stats/team/1610612766',
        "CHI": 'https://www.nba.com/stats/team/1610612741',
        "CLE": 'https://www.nba.com/stats/team/1610612739',
        "DAL": 'https://www.nba.com/stats/team/1610612742',
        "DEN": 'https://www.nba.com/stats/team/1610612743',
        "DET": 'https://www.nba.com/stats/team/1610612765',
        "GSW": 'https://www.nba.com/stats/team/1610612744',
        "HOU": 'https://www.nba.com/stats/team/1610612745',
        "IND": 'https://www.nba.com/stats/team/1610612754',
        "LAC": 'https://www.nba.com/stats/team/1610612746',
        "LAL": 'https://www.nba.com/stats/team/1610612747',
        "MEM": 'https://www.nba.com/stats/team/1610612763',
        "MIA": 'https://www.nba.com/stats/team/1610612748',
        "MIL": 'https://www.nba.com/stats/team/1610612749',
        "MIN": 'https://www.nba.com/stats/team/1610612750',
        "NOP": 'https://www.nba.com/stats/team/1610612740',
        "NYK": 'https://www.nba.com/stats/team/1610612752',
        "OKC": 'https://www.nba.com/stats/team/1610612760',
        "ORL": 'https://www.nba.com/stats/team/1610612753',
        "PHI": 'https://www.nba.com/stats/team/1610612755',
        "PHO": 'https://www.nba.com/stats/team/1610612756',
        "POR": 'https://www.nba.com/stats/team/1610612757',
        "SAC": 'https://www.nba.com/stats/team/1610612758',
        "SAS": 'https://www.nba.com/stats/team/1610612759',
        "TOR": 'https://www.nba.com/stats/team/1610612761',
        "UTA": 'https://www.nba.com/stats/team/1610612762',
        "WAS": 'https://www.nba.com/stats/team/1610612764',
    }

MONTH_TO_NUM = {
        "Jan" : "01",
        "Feb" : "02",
        "Mar" : "03",
        "Apr" : "04",
        "May" : "05",
        "Jun" : "06",
        "Jul" : "07",
        "Aug" : "08",
        "Sep" : "09",
        "Oct" : "10",
        "Nov" : "11",
        "Dec" : "12",
    }

ALL_TEAMS_CONC = dict.fromkeys(TEAM_ABR_TO_URL.keys()) # ABR -> dict(date -> [cum_stats,last10])

## Map teams to NBA stats page
Use selenium to load JS on stats page for a day, getting the last 10 and stats to date for each team in a season
### NOTE: I was being silly, you can scrape this page: https://www.nba.com/stats/teams/advanced?SeasonType=Regular+Season&DateTo=10%2F26%2F2023 its way better

TODO: add PIE at some point?
PIE Formula=(PTS + FGM + FTM – FGA – FTA + Deff.REB + Off.REB/2 + AST + STL + BLK/2 – PF – TO) / (Game.PTS + Game.FGM + Game.FTM – Game.FGA – Game.FTA + Game.Deff.REB + Game.Off.REB/2 + Game.AST + Game.STL + Game.BLK/2 – Game.PF – Game.TO)

This one deprecated for now dont use (or do idk)

In [2]:
def get_stats_thread(info):
    '''
    Go through list of dates and populate ALL_TEAMS_CONC
    dates: list of dates similar to `Tue Oct 29 2013`
    base_url: base url of NBA stats page for this team
    abr: team abbreviation
    RETURNS: dict mapping date -> [cum_stats, last10]
    '''
    dates = info[0]
    base_url = info[1]
    options = Options()
    chrome_ops = webdriver.ChromeOptions()
    chrome_ops.add_argument("--headless=new")
    options.headless = True
    driver = webdriver.Chrome(options=options) # requires appropriate webdriver in PATH variables, or pass path to file as arg `executable_path=`
    # driver.implicitly_wait(2)
    wait = WebDriverWait(driver,10,0.2)
    out_dict = defaultdict(list)
    missing = []
    i = 0
    try:
        for date in dates:
            date_list = date.split()
            team_stats = base_url + f'&DateTo={MONTH_TO_NUM[date_list[1]]}%2F{date_list[2]}%2F{date_list[3]}'
            if i < 82:
                team_stats += '&SeasonType=Regular+Season&Split=lastn'
            else:
                team_stats += '&SeasonType=Playoffs&Split=lastn'
            driver.get(team_stats)
            # load JS
            try:
                wait.until(EC.visibility_of_all_elements_located((By.TAG_NAME, 'td')))
                # wait.until(EC.visibility_of_any_elements_located((By.TAG_NAME, 'table')))
            except Exception as e:
                # print(f'URL: {team_stats} caused exception: {e}')
                missing.append(date)
                continue
            
            html = driver.page_source

            soup = BeautifulSoup(html, 'html.parser')
            results = soup.find_all('table')

            # 24 stats total from i = 1 to 24, only care about 3 - 24
            # GP	MIN	PTS	W	L	WIN%	FGM	FGA	FG%	3PM	3PA	3P%	FTM	FTA	FT%	OREB	DREB	REB	AST	TOV	STL	BLK	PF	+/-
            cumalitive = results[2].find_all('td') # cum stats to this point
            cum_stats = []
            for e in cumalitive[3:]: # only care about last ten games?
                cum_stats.append(float(e.text))

            per_n = results[3].find_all('tr') # every 10 games ranges
            last_10 = []

            # if last 10 range includes at least 5 games, use those, else use previous 10 range
            if float(per_n[-1].find_all('td')[1].text) >= 5 or i < 10 or len(per_n) < 2:
                for e in per_n[-1].find_all('td')[3:]:
                    last_10.append(float(e.text))
                
            else:
                for e in per_n[-2].find_all('td')[3:]:
                    last_10.append(float(e.text))

            out_dict[date].append(cum_stats)
            out_dict[date].append(last_10)
            i += 1
    except Exception as e:
        driver.quit()
        print(e)
        return info[2],out_dict,missing
    print(f'Data for: {info[2]} complete, Captured: {len(out_dict)}, Missing: {len(missing)}')
    driver.quit()
    return info[2],out_dict,missing

Function to call from a ThreadPoolExecutor (if using in a notebook), can called from ProcessPoolExecutor as well (must called from __main__ in an actual Python file)

In [2]:
# TODO: add batch size parameter/fix pooling 
def get_stats_multi_win(info):
    '''
    Go through list of dates and populate ALL_TEAMS_CONC
    dates: list of dates similar to `Tue Oct 29 2013`
    base_url: base url of NBA stats page for this team
    abr: team abbreviation
    RETURNS: dict mapping date -> [cum_stats, last10]
    '''
    dates = info[0]
    base_url = info[1]
    options = Options()
    chrome_ops = webdriver.ChromeOptions()
    chrome_ops.add_argument("--headless=new")
    options.headless = True
    driver = webdriver.Chrome(options=options) # requires appropriate webdriver in PATH variables, or pass path to file as arg `executable_path=`
    out_dict = defaultdict(list)
    missing = []

    try:
        for i in range(4,len(dates),5): # change range step to control how many tabs are opened in a batch, current set to 5
            date_list = []
            k = 0
            for j in range(i-4,i+1):
                curr_date = dates[j].split()
                date_list.append(dates[j])
                team_stats = base_url + f'&DateTo={MONTH_TO_NUM[curr_date[1]]}%2F{curr_date[2]}%2F{curr_date[3]}&Split=lastn'
                if j < 82:
                    team_stats += '&SeasonType=Regular+Season'
                else:
                    team_stats += '&SeasonType=Playoffs'
                driver.get(team_stats)
                k += 1
                if k < 5: # change range step to control how many tabs are opened in a batch, current set to 5
                    driver.execute_script("window.open()")
                    driver.switch_to.window(driver.window_handles[k])
            
            window_list = driver.window_handles
            k = 0
            # load JS
            time.sleep(2)
            for window in window_list:
                date = date_list[k]
                k += 1
                driver.switch_to.window(window)
                html = driver.page_source

                soup = BeautifulSoup(html, 'html.parser')
                results = soup.find_all('table')

                try:
                    # 24 stats total from i = 1 to 24, only care about 3 - 24
                    # GP	MIN	PTS	W	L	WIN%	FGM	FGA	FG%	3PM	3PA	3P%	FTM	FTA	FT%	OREB	DREB	REB	AST	TOV	STL	BLK	PF	+/-
                    cumalitive = results[2].find_all('td') # cum stats to this point
                    cum_stats = []
                    for e in cumalitive[3:]: # only care about last ten games?
                        cum_stats.append(float(e.text))

                    per_n = results[3].find_all('tr') # every 10 games ranges
                    last_10 = []

                    # if last 10 range includes at least 5 games, use those, else use previous 10 range
                    if float(per_n[-1].find_all('td')[1].text) >= 5 or i < 10 or len(per_n) < 2:
                        for e in per_n[-1].find_all('td')[3:]:
                            last_10.append(float(e.text))
                        
                    else:
                        for e in per_n[-2].find_all('td')[3:]:
                            last_10.append(float(e.text))

                    out_dict[date].append(cum_stats)
                    out_dict[date].append(last_10)
                except:
                    missing.append(date)
                if len(driver.window_handles) > 1:
                    driver.close()
        
        date_list = []
        k = 0
        remaining = len(dates) % 5
        if remaining > 0:
            for i in range(len(dates) - remaining, len(dates)):
                curr_date = dates[i].split()
                date_list.append(dates[i])
                team_stats = base_url + f'&DateTo={MONTH_TO_NUM[curr_date[1]]}%2F{curr_date[2]}%2F{curr_date[3]}&Split=lastn'
                if i < 82:
                    team_stats += '&SeasonType=Regular+Season'
                else:
                    team_stats += '&SeasonType=Playoffs'
                driver.get(team_stats)
                k += 1
                if k < remaining:
                    driver.execute_script("window.open()")
                    driver.switch_to.window(driver.window_handles[k])

            
            window_list = driver.window_handles
            k = 0
            # load JS
            time.sleep(2)
            for window in window_list:
                date = date_list[k]
                k += 1
                driver.switch_to.window(window)
                html = driver.page_source

                soup = BeautifulSoup(html, 'html.parser')
                results = soup.find_all('table')

                try:
                    # 24 stats total from i = 1 to 24, only care about 3 - 24
                    # GP	MIN	PTS	W	L	WIN%	FGM	FGA	FG%	3PM	3PA	3P%	FTM	FTA	FT%	OREB	DREB	REB	AST	TOV	STL	BLK	PF	+/-
                    cumalitive = results[2].find_all('td') # cum stats to this point
                    cum_stats = []
                    for e in cumalitive[3:]: # only care about last ten games?
                        cum_stats.append(float(e.text))

                    per_n = results[3].find_all('tr') # every 10 games ranges
                    last_10 = []

                    # if last 10 range includes at least 5 games, use those, else use previous 10 range
                    if float(per_n[-1].find_all('td')[1].text) >= 5 or i < 10 or len(per_n) < 2:
                        for e in per_n[-1].find_all('td')[3:]:
                            last_10.append(float(e.text))
                        
                    else:
                        for e in per_n[-2].find_all('td')[3:]:
                            last_10.append(float(e.text))

                    out_dict[date].append(cum_stats)
                    out_dict[date].append(last_10)
                except:
                    missing.append(date)
                if len(driver.window_handles) > 1:
                    driver.close()

    except Exception as e:
        driver.quit()
        print(f'Data for: {info[2]} complete, Captured: {len(out_dict)}, Missing: {len(missing)}, Error: {e}')
        return info[2],out_dict,missing
    
    print(f'Data for: {info[2]} complete, Captured: {len(out_dict)}, Missing: {len(missing)}')
    driver.quit()
    return info[2],out_dict,missing

WARNING: using multiple threads/processes can put a strain on your system, its also hard to close when it starts running!

In [50]:
import concurrent
import concurrent.futures

def test_this(games_df,szn):
    start_date = games_df['date'][0].split()
    teams = []

    # Generate list of URLS for each team
    for team, abr in TEAM_NAME_TO_ABR.items():
        team_games = games_df.loc[(games_df['away'].str.upper() == team) | (games_df['home'].str.upper() == team)]
        dates = team_games['date'].to_list()
        url = TEAM_ABR_TO_URL[abr] + f'/traditional?Season={szn}&DateFrom={MONTH_TO_NUM[start_date[1]]}%2F{start_date[2]}%2F{start_date[3]}'
        teams.append([dates,url,abr])
        
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(get_stats_multi_win,teams))

    return results

In [52]:
games_df = pd.read_csv('../NBA/games/2021-2022_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])
results = test_this(games_df,'2021-22')

Data for: CHO complete, Captured: 82, Missing: 1
Data for: BOS complete, Captured: 106, Missing: 0
Data for: ATL complete, Captured: 87, Missing: 2
Data for: BRK complete, Captured: 86, Missing: 1
Data for: CHI complete, Captured: 87, Missing: 0
Data for: CLE complete, Captured: 82, Missing: 2
Data for: DET complete, Captured: 82, Missing: 0
Data for: DEN complete, Captured: 86, Missing: 1
list index out of range
Data for: GSW complete, Captured: 104, Missing: 0
Data for: HOU complete, Captured: 82, Missing: 0
Data for: IND complete, Captured: 82, Missing: 0
Data for: LAC complete, Captured: 82, Missing: 2
Data for: LAL complete, Captured: 82, Missing: 0
Data for: MEM complete, Captured: 94, Missing: 0
list index out of range
Data for: MIL complete, Captured: 94, Missing: 0
Data for: MIN complete, Captured: 87, Missing: 2
list index out of range
Data for: NYK complete, Captured: 82, Missing: 0
Data for: OKC complete, Captured: 82, Missing: 0
Data for: ORL complete, Captured: 81, Missin

In [58]:
ALL_TEAMS_CONC = dict.fromkeys(TEAM_ABR_TO_URL.keys()) # ABR -> dict(date -> [cum_stats,last10])

for r in results:
        ALL_TEAMS_CONC[r[0]] = r[1]
        ALL_TEAMS_CONC[r[0]]["MISSED"] = r[2]
    
with open('../NBA/conc_stats/2021-2022_conc_stats.pkl', 'wb') as f:
    pickle.dump(ALL_TEAMS_CONC,f)

# with open('../NBA/conc_stats/2023-2024_conc_stats.pkl', 'rb') as f:
#     save_me = pickle.load(f)

Check if all missing dates are included in "MISSED" key

In [49]:
with open('../NBA/conc_stats/2023-2024_conc_stats.pkl', 'rb') as f:
    conc_2024 = pickle.load(f)

games_df = pd.read_csv('../NBA/games/2023-2024_nba_season.csv',sep=',',names=['date','away','away_pt','home','home_pt'])

def check_missed(conc_stats,games_df):
    unknown_misses = [] # store missed dates that are not in missed key
    for team, abr in TEAM_NAME_TO_ABR.items():
        team_games = games_df.loc[(games_df['away'].str.upper() == team) | (games_df['home'].str.upper() == team)]
        dates = team_games['date'].to_list()
        missing_counted = set(conc_stats[abr]['MISSED'])
        for date in dates:
            if date not in conc_stats[abr] and date not in missing_counted:
                unknown_misses.append(date)
    return unknown_misses

uk_miss = check_missed(conc_2024,games_df)
print(len(uk_miss))

0


Fixing data

In [19]:
def get_missing(info):
    '''
    Function to retry missing dates
    expects list with args:
    [dates,base_url,abr,reg_szn_end,play_in_end]
    reg_szn_end: date of final regular season game
    play_in_end: date of final play in game (empty if no play in)
    returns a dict mapping dates to stats
    '''
    dates = info[0]
    base_url = info[1]
    reg_szn_end = info[3]
    play_in_end = info[4]
    options = Options()
    driver = webdriver.Chrome(options=options) # requires appropriate webdriver in PATH variables, or pass path to file as arg `executable_path=`
    out_dict = defaultdict(list)
    missing = []

    try:
        for i in range(len(dates)):
            date_list = []
            curr_date = dates[i].split()
            date_list.append(dates[i])
            team_stats = base_url + f'&DateTo={MONTH_TO_NUM[curr_date[1]]}%2F{curr_date[2]}%2F{curr_date[3]}&Split=lastn'

            if curr_date[3] < reg_szn_end[3] or (MONTH_TO_NUM[curr_date[1]] <= reg_szn_end[1] and curr_date[2] <= reg_szn_end[2]): # reg szn tourny??
                team_stats += '&SeasonType=Regular+Season'
            elif len(play_in_end) > 0 and curr_date[1] <= play_in_end[1] and curr_date[2] <= play_in_end[2]:
                team_stats += '&SeasonType=PlayIn'
            else:
                team_stats += '&SeasonType=Playoffs'

            driver.get(team_stats)

            if i < len(dates) - 1: # change range step to control how many tabs are opened in a batch, current set to 5
                driver.execute_script("window.open()")
                driver.switch_to.window(driver.window_handles[k])
            
            window_list = driver.window_handles
            k = 0
            # load JS
            time.sleep(5)
            for window in window_list:
                date = date_list[k]
                k += 1
                driver.switch_to.window(window)
                html = driver.page_source

                soup = BeautifulSoup(html, 'html.parser')
                results = soup.find_all('table')

                try:
                    # 24 stats total from i = 1 to 24, only care about 3 - 24
                    # GP	MIN	PTS	W	L	WIN%	FGM	FGA	FG%	3PM	3PA	3P%	FTM	FTA	FT%	OREB	DREB	REB	AST	TOV	STL	BLK	PF	+/-
                    cumalitive = results[2].find_all('td') # cum stats to this point
                    cum_stats = []
                    for e in cumalitive[3:]: # only care about last ten games?
                        cum_stats.append(float(e.text))

                    per_n = results[3].find_all('tr') # every 10 games ranges
                    last_10 = []

                    # if last 10 range includes at least 5 games, use those, else use previous 10 range
                    if float(per_n[-1].find_all('td')[1].text) >= 5 or i < 10 or len(per_n) < 2:
                        for e in per_n[-1].find_all('td')[3:]:
                            last_10.append(float(e.text))
                        
                    else:
                        for e in per_n[-2].find_all('td')[3:]:
                            last_10.append(float(e.text))

                    out_dict[date].append(cum_stats)
                    out_dict[date].append(last_10)
                except:
                    missing.append(date)
                if len(driver.window_handles) > 1:
                    driver.close()
        

    except Exception as e:
        driver.quit()
        print(f'Data for: {info[2]} complete, Captured: {len(out_dict)}, Missing: {len(missing)}, Error: {e}')
        return info[2],out_dict,missing
    
    print(f'Data for: {info[2]} complete, Captured: {len(out_dict)}, Missing: {len(missing)}')
    driver.quit()
    return info[2],out_dict,missing

In [20]:
import concurrent
import concurrent.futures

def retry_missing(games_df,conc_stats,szn,reg_szn_end,play_in_end):
    start_date = games_df['date'][0].split()
    teams = []

    # Generate list of URLS for each team
    for team, abr in TEAM_NAME_TO_ABR.items():
        missing = conc_stats[abr]['MISSED']
        if len(missing) == 0:
            continue
        url = TEAM_ABR_TO_URL[abr] + f'/traditional?Season={szn}&DateFrom={MONTH_TO_NUM[start_date[1]]}%2F{start_date[2]}%2F{start_date[3]}'
        teams.append([missing,url,abr,reg_szn_end,play_in_end])
        
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(get_stats_multi_win,teams))

    return results

In [43]:
with open('../NBA/conc_stats/2022-2023_conc_stats.pkl', 'rb') as f:
    conc_2023 = pickle.load(f)

games_df_2023 = pd.read_csv('../NBA/games/2022-2023_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])

reg_szn_end_2023 = ["","04","9","2023"]
play_in_end_2023 = ["","04","14","2023"]

res_2023 = retry_missing(games_df_2023,conc_2023,'2022-23',reg_szn_end_2023,play_in_end_2023)

with open('../NBA/conc_stats/2021-2022_conc_stats.pkl', 'rb') as f:
    conc_2022 = pickle.load(f)

games_df_2022 = pd.read_csv('../NBA/games/2021-2022_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])

reg_szn_end_2022 = ["","04","10","2022"]
play_in_end_2022 = ["","04","15","2022"]

res_2022 = retry_missing(games_df_2022,conc_2022,'2021-22',reg_szn_end_2022,play_in_end_2022)

with open('../NBA/conc_stats/2020-2021_conc_stats.pkl', 'rb') as f:
    conc_2021 = pickle.load(f)

games_df_2021 = pd.read_csv('../NBA/games/2020-2021_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])

reg_szn_end_2021 = ["","05","16","2021"]
play_in_end_2021 = ["","05","21","2021"]

res_2021 = retry_missing(games_df_2021,conc_2021,'2020-21',reg_szn_end_2021,play_in_end_2021)

with open('../NBA/conc_stats/2019-2020_conc_stats.pkl', 'rb') as f:
    conc_2020 = pickle.load(f)

games_df_2020 = pd.read_csv('../NBA/games/2019-2020_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])

reg_szn_end_2020 = ["","08","14","2020"]
play_in_end_2020 = ["","08","15","2020"]

res_2020 = retry_missing(games_df_2020,conc_2020,'2019-20',reg_szn_end_2020,play_in_end_2020)

with open('../NBA/conc_stats/2018-2019_conc_stats.pkl', 'rb') as f:
    conc_2019 = pickle.load(f)

games_df_2019 = pd.read_csv('../NBA/games/2018-2019_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])

reg_szn_end_2019 = ["","04","10","2019"]
play_in_end_2019 = []

res_2019 = retry_missing(games_df_2019,conc_2019,'2018-19',reg_szn_end_2019,play_in_end_2019)

with open('../NBA/conc_stats/2017-2018_conc_stats.pkl', 'rb') as f:
    conc_2018 = pickle.load(f)

games_df_2018 = pd.read_csv('../NBA/games/2017-2018_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])

reg_szn_end_2018 = ["","04","11","2018"]
play_in_end_2018 = []

res_2018 = retry_missing(games_df_2018,conc_2018,'2017-18',reg_szn_end_2018,play_in_end_2018)

with open('../NBA/conc_stats/2016-2017_conc_stats.pkl', 'rb') as f:
    conc_2017 = pickle.load(f)

games_df_2017 = pd.read_csv('../NBA/games/2016-2017_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])

reg_szn_end_2017 = ["","04","12","2017"]
play_in_end_2017 = []

res_2017 = retry_missing(games_df_2017,conc_2017,'2016-17',reg_szn_end_2017,play_in_end_2017)

# MISSED
# res_2022 = retry_missing(games_df,conc_2022,'2022-23',reg_szn_end_2023,play_in_end_2023)

Data for: WAS complete, Captured: 1, Missing: 0
Data for: CHI complete, Captured: 2, Missing: 0
Data for: PHO complete, Captured: 2, Missing: 0
Data for: UTA complete, Captured: 2, Missing: 0
Data for: NYK complete, Captured: 2, Missing: 0
Data for: GSW complete, Captured: 2, Missing: 0
Data for: NYK complete, Captured: 2, Missing: 0
Data for: POR complete, Captured: 3, Missing: 0
Data for: WAS complete, Captured: 2, Missing: 0
Data for: DAL complete, Captured: 1, Missing: 0
Data for: DEN complete, Captured: 3, Missing: 0


In [44]:
res_list = [[res_2017,conc_2017],[res_2018,conc_2018],[res_2019,conc_2019],[res_2020,conc_2020],[res_2021,conc_2021],[res_2022,conc_2022],[res_2023,conc_2023]]
for res,conc in res_list:
    for r in res:
        if len(r[1]) != len(conc[r[0]]['MISSED']): # make sure we got all missing
            print(r[1])
            continue
        conc[r[0]]['MISSED'] = []
        for d,f in r[1].items():
            conc[r[0]][d] = f
        

In [45]:
with open('../NBA/conc_stats/2022-2023_conc_stats.pkl', 'wb') as f:
    pickle.dump(conc_2023,f)

with open('../NBA/conc_stats/2021-2022_conc_stats.pkl', 'wb') as f:
    pickle.dump(conc_2022,f)

with open('../NBA/conc_stats/2020-2021_conc_stats.pkl', 'wb') as f:
    pickle.dump(conc_2021,f)

with open('../NBA/conc_stats/2019-2020_conc_stats.pkl', 'wb') as f:
    pickle.dump(conc_2020,f)
    
with open('../NBA/conc_stats/2018-2019_conc_stats.pkl', 'wb') as f:
    pickle.dump(conc_2019,f)

with open('../NBA/conc_stats/2017-2018_conc_stats.pkl', 'wb') as f:
    pickle.dump(conc_2018,f)

with open('../NBA/conc_stats/2016-2017_conc_stats.pkl', 'wb') as f:
    pickle.dump(conc_2017,f)


In [50]:
check_list = [[games_df_2017,conc_2017],[games_df_2018,conc_2018],[games_df_2019,conc_2019],[games_df_2020,conc_2020],[games_df_2021,conc_2021],[games_df_2022,conc_2022],[games_df_2023,conc_2023]]
for games,conc in check_list:
    uk_miss = check_missed(conc,games)
    print(len(uk_miss))

0
0
0
0
0
0
0


In [34]:
for r in res:
    if len(r[1]) != len(conc_2024[r[0]]['MISSED']): # make sure we got all missing
        print(r[1])
        continue
    conc_2024[r[0]]['MISSED'] = []
    for d,f in r[1].items():
        conc_2024[r[0]][d] = f
    
# with open('../NBA/conc_stats/2023-2024_conc_stats.pkl', 'wb') as f:
#     pickle.dump(conc_2024,f)

In [38]:
with open('../NBA/conc_stats/2022-2023_conc_stats.pkl', 'wb') as f:
    pickle.dump(conc_2024,f)

In [33]:
import datetime
# TODO: add batch size parameter/fix pooling 
def get_adv_stats(info):
    '''
    Gets cumulative advanced stats for a given season, NOT INCLUDING LAST 10
    input: [dates,url,abr,reg_szn_end,play_in_end]
    RETURNS: dict mapping date -> [abr,adv_stats]
    '''
    dates = info[0]
    base_url = info[1]
    reg_szn_end = info[3]
    play_in_end = info[4]
    reg_szn_end_dt = datetime.datetime(int(reg_szn_end[3]),int(reg_szn_end[1]),int(reg_szn_end[2]))
    if len(play_in_end) > 0:
        play_in_end_dt = datetime.datetime(int(play_in_end[3]),int(play_in_end[1]),int(play_in_end[2]))

    options = Options()
    driver = webdriver.Chrome(options=options) # requires appropriate webdriver in PATH variables, or pass path to file as arg `executable_path=`
    # teams_dict = info[2]
    out_dict = defaultdict(list) # map date -> [abr,adv_stats]
    missing = []

    try:
        for i in range(4,len(dates),5): # change range step to control how many tabs are opened in a batch, current set to 5
            date_list = []
            k = 0
            for j in range(i-4,i+1):
                curr_date = dates[j].split()
                date_list.append(dates[j])
                team_stats = base_url + f'&DateTo={MONTH_TO_NUM[curr_date[1]]}%2F{curr_date[2]}%2F{curr_date[3]}'

                curr_date_dt = datetime.datetime(int(curr_date[3]),int(MONTH_TO_NUM[curr_date[1]]),int(curr_date[2]))

                if curr_date_dt <= reg_szn_end_dt:
                    team_stats += '&SeasonType=Regular+Season'
                elif len(play_in_end) > 0 and curr_date_dt <= play_in_end_dt:
                    team_stats += '&SeasonType=PlayIn'
                else:
                    team_stats += '&SeasonType=Playoffs'

                driver.get(team_stats)
                k += 1
                if k < 5: # change range step to control how many tabs are opened in a batch, current set to 10
                    driver.execute_script("window.open()")
                    driver.switch_to.window(driver.window_handles[k])
            
            window_list = driver.window_handles
            k = 0
            # load JS
            time.sleep(2)
            for window in window_list:
                date = date_list[k]
                k += 1
                driver.switch_to.window(window)
                html = driver.page_source

                soup = BeautifulSoup(html, 'html.parser')
                results = soup.find_all('table')

                try:
                    teams = results[2].find_all('tr')
                    for team in teams[1:]:
                        # rank 	TEAM	GP	W	L	MIN	OFFRTG	DEFRTG	NETRTG	AST%	AST/TO	ASTRATIO	OREB%	DREB%	REB%	TOV%	EFG%	TS%	PACE	PIE	POS
                        # [6:20]
                        adv = []
                        stats = team.find_all('td')
                        abr = TEAM_NAME_TO_ABR[stats[1].text.upper()]
                        for e in stats[6:20]:
                            adv.append(float(e.text))

                        out_dict[date].append([abr,adv])

                except Exception as e:
                    print(f'Exception: {e} for team: {abr} on link: {team_stats}')
                    missing.append(date)
                if len(driver.window_handles) > 1:
                    driver.close()
        
        date_list = []
        k = 0
        remaining = len(dates) % 5
        if remaining > 0:
            for i in range(len(dates) - remaining, len(dates)):
                curr_date = dates[i].split()
                date_list.append(dates[i])
                team_stats = base_url + f'&DateTo={MONTH_TO_NUM[curr_date[1]]}%2F{curr_date[2]}%2F{curr_date[3]}'

                curr_date_dt = datetime.datetime(int(curr_date[3]),int(MONTH_TO_NUM[curr_date[1]]),int(curr_date[2]))

                if curr_date_dt <= reg_szn_end_dt:
                    team_stats += '&SeasonType=Regular+Season'
                elif len(play_in_end) > 0 and curr_date_dt <= play_in_end_dt:
                    team_stats += '&SeasonType=PlayIn'
                else:
                    team_stats += '&SeasonType=Playoffs'

                driver.get(team_stats)
                k += 1
                if k < remaining:
                    driver.execute_script("window.open()")
                    driver.switch_to.window(driver.window_handles[k])

            
            window_list = driver.window_handles
            k = 0
            # load JS
            time.sleep(2)
            for window in window_list:
                date = date_list[k]
                k += 1
                driver.switch_to.window(window)
                html = driver.page_source

                soup = BeautifulSoup(html, 'html.parser')
                results = soup.find_all('table')

                try:
                    teams = results[2].find_all('tr')
                    for team in teams[1:]:
                        # rank 	TEAM	GP	W	L	MIN	OFFRTG	DEFRTG	NETRTG	AST%	AST/TO	ASTRATIO	OREB%	DREB%	REB%	TOV%	EFG%	TS%	PACE	PIE	POS
                        # [6:20]
                        adv = []
                        stats = team.find_all('td')
                        abr = TEAM_NAME_TO_ABR[stats[1].text.upper()]
                        for e in stats[6:20]:
                            adv.append(float(e.text))

                        out_dict[date].append([abr,adv])

                except Exception as e:
                    print(f'Exception: {e} for team: {abr} on link: {team_stats}')
                    missing.append(date)
                if len(driver.window_handles) > 1:
                    driver.close()

    except Exception as e:
        driver.quit()
        print(f'Captured: {len(out_dict)}, Missing: {len(missing)}, Error: {e}')
        return out_dict,missing
    
    print(f'Captured: {len(out_dict)}, Missing: {len(missing)}')
    driver.quit()
    return out_dict,missing

In [17]:
import concurrent
import concurrent.futures

def get_adv_pool(df_list,szn,reg_szn_end,play_in_end):
    pack = []
    for i in range(len(df_list)):
        start_date = df_list[i]['date'][0].split()
        dates = df_list[i]['date'].unique() # Get unique dates as list
        pack.append([dates,f'https://www.nba.com/stats/teams/advanced?Season={szn[i]}&DateFrom={MONTH_TO_NUM[start_date[1]]}%2F{start_date[2]}%2F{start_date[3]}','',reg_szn_end[i],play_in_end[i]])

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(get_adv_stats,pack))

    return results

In [9]:
df_list = []
szn_list = []
reg_szn_list = []
play_in_list = []

games_df_2024 = pd.read_csv('../NBA/games/2023-2024_nba_season.csv',sep=',',names=['date','away','away_pt','home','home_pt'])
reg_szn_end_2024 = ["","04","14","2024"]
play_in_end_2024 = ["","04","19","2024"]

df_list.append(games_df_2024)
reg_szn_list.append(reg_szn_end_2024)
play_in_list.append(play_in_end_2024)
szn_list.append('2023-24')

games_df_2023 = pd.read_csv('../NBA/games/2022-2023_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])
reg_szn_end_2023 = ["","04","9","2023"]
play_in_end_2023 = ["","04","14","2023"]

df_list.append(games_df_2023)
reg_szn_list.append(reg_szn_end_2023)
play_in_list.append(play_in_end_2023)
szn_list.append('2022-23')

games_df_2022 = pd.read_csv('../NBA/games/2021-2022_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])
reg_szn_end_2022 = ["","04","10","2022"]
play_in_end_2022 = ["","04","15","2022"]

df_list.append(games_df_2022)
reg_szn_list.append(reg_szn_end_2022)
play_in_list.append(play_in_end_2022)
szn_list.append('2021-22')

games_df_2021 = pd.read_csv('../NBA/games/2020-2021_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])
reg_szn_end_2021 = ["","05","16","2021"]
play_in_end_2021 = ["","05","21","2021"]

df_list.append(games_df_2021)
reg_szn_list.append(reg_szn_end_2021)
play_in_list.append(play_in_end_2021)
szn_list.append('2020-21')

games_df_2020 = pd.read_csv('../NBA/games/2019-2020_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])
reg_szn_end_2020 = ["","08","14","2020"]
play_in_end_2020 = ["","08","15","2020"]

df_list.append(games_df_2020)
reg_szn_list.append(reg_szn_end_2020)
play_in_list.append(play_in_end_2020)
szn_list.append('2019-20')

games_df_2019 = pd.read_csv('../NBA/games/2018-2019_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])
reg_szn_end_2019 = ["","04","10","2019"]
play_in_end_2019 = []

df_list.append(games_df_2019)
reg_szn_list.append(reg_szn_end_2019)
play_in_list.append(play_in_end_2019)
szn_list.append('2018-19')

games_df_2018 = pd.read_csv('../NBA/games/2017-2018_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])
reg_szn_end_2018 = ["","04","11","2018"]
play_in_end_2018 = []

df_list.append(games_df_2018)
reg_szn_list.append(reg_szn_end_2018)
play_in_list.append(play_in_end_2018)
szn_list.append('2017-18')

games_df_2017 = pd.read_csv('../NBA/games/2016-2017_season_inj.csv',sep=',',names=['date','away','away_pt','home','home_pt'])
reg_szn_end_2017 = ["","04","12","2017"]
play_in_end_2017 = []

df_list.append(games_df_2017)
reg_szn_list.append(reg_szn_end_2017)
play_in_list.append(play_in_end_2017)
szn_list.append('2016-17')

In [21]:
res = get_adv_pool([games_df_2023],['2022-23'],[reg_szn_end_2023],[play_in_end_2023])
res_2023 = res

Captured: 212, Missing: 0


In [36]:
res_tot = get_adv_pool(df_list,szn_list,reg_szn_list,play_in_list)

Captured: 212, Missing: 0
Captured: 193, Missing: 0
Captured: 204, Missing: 0
Captured: 203, Missing: 0
Captured: 213, Missing: 0
Captured: 191, Missing: 0
Captured: 212, Missing: 0
Captured: 213, Missing: 0


In [40]:
for i in range(len(res_tot)):
    curr_dict = res_tot[i][0]
    with open(f'../NBA/conc_stats_adv/{szn_list[i]}_conc_stats.pkl', 'wb') as f:
        pickle.dump(curr_dict,f)


In [ ]:
# https://www.nba.com/stats/team/1610612738/onoffcourt-advanced
driver = webdriver.Chrome()
driver.get("https://www.nba.com/stats/team/1610612738/onoffcourt-advanced")
time.sleep(5)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
results = soup.find_all('table') # 2-4, overall, on, off, find_all('tr')[1:], <- .find_all('td') where i = 0 has Last, First,
overall = results[2].find_all('tr')[1].find_all('td')
on_court = results[3].find_all('tr')[1:]
off_court = results[4].find_all('tr')[1:]

# 	GP	MIN	OFFRTG	DEFRTG	NETRTG	AST%	AST/TO	AST RATIO	OREB%	DREB%	REB%	TOV%	EFG%	TS%	PACE	PIE

In [10]:
import datetime
# TODO: add batch size parameter/fix pooling 
def get_on_off(info):
    '''
    Gets cumulative advanced stats for a given season, NOT INCLUDING LAST 10
    input: [dates,url,abr,reg_szn_end,play_in_end]
    RETURNS: dict mapping date -> [abr,adv_stats]
    '''
    dates = info[0]
    base_url = info[1]
    abr = info[2]
    reg_szn_end = info[3]
    play_in_end = info[4]
    reg_szn_end_dt = datetime.datetime(int(reg_szn_end[3]),int(reg_szn_end[1]),int(reg_szn_end[2]))
    if len(play_in_end) > 0:
        play_in_end_dt = datetime.datetime(int(play_in_end[3]),int(play_in_end[1]),int(play_in_end[2]))

    options = Options()
    driver = webdriver.Chrome(options=options) # requires appropriate webdriver in PATH variables, or pass path to file as arg `executable_path=`
    # teams_dict = info[2]
    out_dict = defaultdict(list) # map date -> [abr,adv_stats]
    missing = []

    try:
        for i in range(9,len(dates),10): # change range step to control how many tabs are opened in a batch, current set to 5
            date_list = []
            k = 0
            for j in range(i-9,i+1):
                curr_date = dates[j].split()
                date_list.append(dates[j])
                team_stats = base_url + f'&DateTo={MONTH_TO_NUM[curr_date[1]]}%2F{curr_date[2]}%2F{curr_date[3]}'

                curr_date_dt = datetime.datetime(int(curr_date[3]),int(MONTH_TO_NUM[curr_date[1]]),int(curr_date[2]))

                if curr_date_dt <= reg_szn_end_dt:
                    team_stats += '&SeasonType=Regular+Season'
                elif len(play_in_end) > 0 and curr_date_dt <= play_in_end_dt:
                    team_stats += '&SeasonType=PlayIn'
                else:
                    team_stats += '&SeasonType=Playoffs'

                driver.get(team_stats)
                k += 1
                if k < 10: # change range step to control how many tabs are opened in a batch, current set to 10
                    driver.execute_script("window.open()")
                    driver.switch_to.window(driver.window_handles[k])
            
            window_list = driver.window_handles
            k = 0
            # load JS
            time.sleep(2)
            for window in window_list:
                date = date_list[k]
                k += 1
                driver.switch_to.window(window)
                html = driver.page_source

                soup = BeautifulSoup(html, 'html.parser')
                results = soup.find_all('table')

                try:
                    overall = results[2].find_all('tr')[1].find_all('td')
                    over_stats = []
                    on_dict = defaultdict(list)
                    off_dict = defaultdict(list)

                    for over in overall[1:]:
                        over_stats.append(float(over.text))

                    out_dict[date].append(over_stats)

                    # NAME	GP	MIN	OFFRTG	DEFRTG	NETRTG	AST%	AST/TO	AST RATIO	OREB%	DREB%	REB%	TOV%	EFG%	TS%	PACE	PIE
                    on_court = results[3].find_all('tr')[1:]
                    for player in on_court:
                        on_stats = []
                        player_stats = player.find_all('td')
                        name_strip = player_stats[0].text
                        d = ''.join(name_strip.split())
                        full_name = ' '.join(d.split(',')[::-1]).upper()
                        for e in player_stats[1:]:
                            on_stats.append(float(e.text))
                        on_dict[full_name] = on_stats

                    out_dict[date].append(on_dict)

                    off_court = results[4].find_all('tr')[1:]
                    for player in off_court:
                        off_stats = []
                        player_stats = player.find_all('td')
                        name_strip = player_stats[0].text
                        d = ''.join(name_strip.split())
                        full_name = ' '.join(d.split(',')[::-1]).upper()
                        for e in player_stats[1:]:
                            off_stats.append(float(e.text))
                        off_dict[full_name] = off_stats

                    out_dict[date].append(off_dict)

                except Exception as e:
                    print(f'Exception: {e} for team: {info[2]} on link: {team_stats}')
                    missing.append(date)
                if len(driver.window_handles) > 1:
                    driver.close()
        
        date_list = []
        k = 0
        remaining = len(dates) % 10
        if remaining > 0:
            for i in range(len(dates) - remaining, len(dates)):
                curr_date = dates[i].split()
                date_list.append(dates[i])
                team_stats = base_url + f'&DateTo={MONTH_TO_NUM[curr_date[1]]}%2F{curr_date[2]}%2F{curr_date[3]}'

                curr_date_dt = datetime.datetime(int(curr_date[3]),int(MONTH_TO_NUM[curr_date[1]]),int(curr_date[2]))

                if curr_date_dt <= reg_szn_end_dt:
                    team_stats += '&SeasonType=Regular+Season'
                elif len(play_in_end) > 0 and curr_date_dt <= play_in_end_dt:
                    team_stats += '&SeasonType=PlayIn'
                else:
                    team_stats += '&SeasonType=Playoffs'

                driver.get(team_stats)
                k += 1
                if k < remaining:
                    driver.execute_script("window.open()")
                    driver.switch_to.window(driver.window_handles[k])

            
            window_list = driver.window_handles
            k = 0
            # load JS
            time.sleep(2)
            for window in window_list:
                date = date_list[k]
                k += 1
                driver.switch_to.window(window)
                html = driver.page_source

                soup = BeautifulSoup(html, 'html.parser')
                results = soup.find_all('table')

                try:
                    overall = results[2].find_all('tr')[1].find_all('td')
                    over_stats = []
                    on_dict = defaultdict(list)
                    off_dict = defaultdict(list)

                    for over in overall[1:]:
                        over_stats.append(float(over.text))

                    out_dict[date].append(over_stats)

                    # NAME	GP	MIN	OFFRTG	DEFRTG	NETRTG	AST%	AST/TO	AST RATIO	OREB%	DREB%	REB%	TOV%	EFG%	TS%	PACE	PIE
                    on_court = results[3].find_all('tr')[1:]
                    for player in on_court:
                        on_stats = []
                        player_stats = player.find_all('td')
                        name_strip = player_stats[0].text
                        d = ''.join(name_strip.split())
                        full_name = ' '.join(d.split(',')[::-1]).upper()
                        for e in player_stats[1:]:
                            on_stats.append(float(e.text))
                        on_dict[full_name] = on_stats

                    out_dict[date].append(on_dict)

                    off_court = results[4].find_all('tr')[1:]
                    for player in off_court:
                        off_stats = []
                        player_stats = player.find_all('td')
                        name_strip = player_stats[0].text
                        d = ''.join(name_strip.split())
                        full_name = ' '.join(d.split(',')[::-1]).upper()
                        for e in player_stats[1:]:
                            off_stats.append(float(e.text))
                        off_dict[full_name] = off_stats

                    out_dict[date].append(off_dict)

                except Exception as e:
                    print(f'Exception: {e} for team: {abr} on link: {team_stats}')
                    missing.append(date)
                if len(driver.window_handles) > 1:
                    driver.close()

    except Exception as e:
        driver.quit()
        print(f'Team: {info[2]}, Captured: {len(out_dict)}, Missing: {len(missing)}, Error: {e}')
        return info[2],out_dict,missing
    
    print(f'Team: {info[2]}, Captured: {len(out_dict)}, Missing: {len(missing)}')
    driver.quit()
    return info[2],out_dict,missing

In [11]:
import concurrent
import concurrent.futures

def pop_on_off(games_df,szn,reg_szn,play_in):
    start_date = games_df['date'][0].split()
    teams = []

    # Generate list of URLS for each team
    for team, abr in TEAM_NAME_TO_ABR.items():
        team_games = games_df.loc[(games_df['away'].str.upper() == team) | (games_df['home'].str.upper() == team)]
        dates = team_games['date'].to_list()
        url = TEAM_ABR_TO_URL[abr] + f'/onoffcourt-advanced?Season={szn}&DateFrom={MONTH_TO_NUM[start_date[1]]}%2F{start_date[2]}%2F{start_date[3]}'
        teams.append([dates,url,abr,reg_szn,play_in]) #regszn end, playin end
        
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(get_on_off,teams))

    return results

In [12]:
res = pop_on_off(games_df_2024,'2023-24',reg_szn_end_2024,play_in_end_2024)

Exception: list index out of range for team: CHO on link: https://www.nba.com/stats/team/1610612766/onoffcourt-advanced?Season=2023-24&DateFrom=10%2F24%2F2023&DateTo=11%2F14%2F2023&SeasonType=Regular+Season
Exception: list index out of range for team: CHO on link: https://www.nba.com/stats/team/1610612766/onoffcourt-advanced?Season=2023-24&DateFrom=10%2F24%2F2023&DateTo=12%2F8%2F2023&SeasonType=Regular+Season
Exception: list index out of range for team: CHI on link: https://www.nba.com/stats/team/1610612741/onoffcourt-advanced?Season=2023-24&DateFrom=10%2F24%2F2023&DateTo=02%2F14%2F2024&SeasonType=Regular+Season
Team: CHI, Captured: 83, Missing: 1
Team: CHO, Captured: 78, Missing: 2, Error: name 'abr' is not defined
Exception: list index out of range for team: BOS on link: https://www.nba.com/stats/team/1610612738/onoffcourt-advanced?Season=2023-24&DateFrom=10%2F24%2F2023&DateTo=03%2F12%2F2024&SeasonType=Regular+Season
Team: ATL, Captured: 83, Missing: 0
Team: BRK, Captured: 82, Missin

In [14]:
print('grug')

grug


In [15]:
on_off_dict = dict.fromkeys(TEAM_ABR_TO_URL.keys()) # ABR -> dict(date -> [cum_stats,last10])

for r in res:
    on_off_dict[r[0]] = r[1]
    on_off_dict[r[0]]["MISSED"] = r[2]
    
with open('../NBA/conc_on_off/2023-2024_conc_stats.pkl', 'wb') as f:
    pickle.dump(on_off_dict,f)

# with open('../NBA/conc_stats/2023-2024_conc_stats.pkl', 'rb') as f:
#     save_me = pickle.load(f)

In [4]:
with open('../NBA/on_off_stats/2023-2024_on_off.pkl', 'rb') as f:
    on_off_2023_2024 = pickle.load(f)
with open('../NBA/on_off_stats/2023-2024_team_stats.pkl', 'rb') as f:
    team_stats_2023_2024 = pickle.load(f)
    
nba_szn_2023_2024 = Nba_Season('2023','2024',team_stats=team_stats_2023_2024,team_on_off=on_off_2023_2024)
inj = {'DAL': ['OLIVIER-MAXENCE PROSPER'], 'BOS': []}
home_stats,away_stats = nba_szn_2023_2024.calc_injury_impact(inj, 'BOS', 'DAL')
greg = np.subtract(away_stats,home_stats)

In [5]:
nba_szn_2023_2024.add_bet_info('test_today.csv','test_today_with_bet.csv')

[['Thu Jun 6 2024',
  'Dallas Mavericks',
  '107',
  'Boston Celtics',
  '119',
  '-6.5 (-106)',
  'o214.5 (-112)',
  '-250',
  '+6.5 (-114)',
  'u214.5 (-108)',
  '+205']]